In [1]:
include("../src/Julia.jl")

parseCPPData (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [11]:
graphp = "../../graphs/pathDisjoint_1000_exp20/"

"../../graphs/pathDisjoint_1000_exp20/"

In [12]:
a = readFromFile(graphp * "graph.mtx");

In [13]:
tree = []
tr = readFromFile(graphp * "tree0.mtx");
push!(tree, tr);

In [14]:
x = readFromFile(graphp * "x.vec");

In [15]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [17]:
numIts = 300;

@time for i in 1:1
    @time F = treeSolver(tree[i])
    @time F(b);
    
    # a run on standard Float64 data types
    @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
    norm(lap(a) * myx64 - b) / norm(b)
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_tree$(i)_default.txt", dbg)
    
    # now useing higer precision data types
    for hp in [64,128,256,512,1024]
        set_bigfloat_precision(hp)
        laHP = toHighPrecision(la; precision=hp)
        treeHP = toHighPrecision(tree[i]; precision=hp)
        xHP = toHighPrecision(x; precision=hp);
        bHP = toHighPrecision(b; precision=hp);
        FHP = treeSolver(treeHP)

        @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
        insert!(dbg, 1, "$(numIts) iterations")
        insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
        writeToFile(graphp * "_log_julia_tree$(i)_$(hp).txt", dbg)
    end
end

  0.000339 seconds (129 allocations: 374.813 KB)
  0.001524 seconds (17.96 k allocations: 344.375 KB)
  0.289437 seconds (5.42 M allocations: 136.131 MB, 14.12% gc time)
Finished iteration 1 with errors 0.5008401507327659 0.3275599229923137
Finished iteration 11 with errors 0.08600692092716472 0.026209348985554975
Finished iteration 21 with errors 0.04434816393372275 0.008625360570549434
Finished iteration 31 with errors 0.029729653409403953 0.004059662956546587
Finished iteration 41 with errors 0.021628885875451524 0.0021553611358799
Finished iteration 51 with errors 0.0175395058206024 0.0027183421699860325
Finished iteration 61 with errors 0.014290848364793503 0.0010893587488395925
Finished iteration 71 with errors 0.012157545268106291 0.0005946592576459946
Finished iteration 81 with errors 0.010216502668171768 0.0007024418065432016
Finished iteration 91 with errors 0.009034392174101817 0.0006147131891326411
Finished iteration 101 with errors 0.007360330997935377 0.000396722905651905